# Apprentissage LUIS (Language Understanding Intelligence Service)

helper : https://docs.microsoft.com/en-us/azure/bot-service/bot-builder-howto-v4-luis?view=azure-bot-service-4.0&tabs=python

In [3]:
import time
import os

from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from msrest.authentication import CognitiveServicesCredentials

AUTHORING_KEY_LUIS = os.environ["AUTHORING_KEY_LUIS"]
AUTHORING_ENDPOINT_LUIS = os.environ["AUTHORING_ENDPOINT_LUIS"]
# Instantiate a LUIS client
client = LUISAuthoringClient(
    AUTHORING_ENDPOINT_LUIS, CognitiveServicesCredentials(AUTHORING_KEY_LUIS))

In [2]:
app_id = "8958f321-de6e-4cee-bf26-e2468b5f381f"

In [3]:

app_name = "Fly Bot"
app_desc = "Flight booking app built with LUIS Python SDK."
app_version = "0.1"
app_locale = "en-us"

# app_id = client.apps.add(dict(name=app_name,
#                                   initial_version_id=app_version,
#                                   description=app_desc,
#                                   culture=app_locale))

client.apps.get(app_id=app_id)

print("Created LUIS app {}\n    with ID {}".format(app_name, app_id))

Created LUIS app Fly Bot
    with ID 8958f321-de6e-4cee-bf26-e2468b5f381f


In [4]:
information = client.apps.get(app_id)

app_version = information.active_version
app_locale = information.culture
app_desc = information.description
app_name = information.name

In [3]:
from luis_module.get_data import LuisData 

luis_data = LuisData("./data/frames.json")
luis_data.load_data()

In [5]:

luis_transformed_data = luis_data.transform_data()

In [27]:
def add_entities(app_id, app_version):
    '''Création des entitées :
        - 'dst_city'
        - 'or_city'
        - 'str_date'
        - 'end_date'
        - 'n_adult'
        - 'n_children'
        - 'budget'
        - 'seat'
            
            Parameters:
                    app_id : ID de l\'application 
                    app_version : version de l\'application LUIS
            Returns:
                    None
    '''
    dst_city_entity_id = client.model.add_entity(
        app_id, app_version, name="dst_city")
    print("destinationEntityId {} added.".format(dst_city_entity_id))

    or_city_entity_id = client.model.add_entity(
        app_id, app_version, name="or_city")
    print("classEntityId {} added.".format(or_city_entity_id))

    str_date_entity_id = client.model.add_entity(
        app_id, app_version, name="str_date")
    print("flightEntityId {} added.".format(str_date_entity_id))

    end_date_entity_id = client.model.add_entity(
        app_id, app_version, name="end_date")
    print("flightEntityId {} added.".format(end_date_entity_id))

    n_adult_entity_id = client.model.add_entity(
        app_id, app_version, name="n_adult")
    print("flightEntityId {} added.".format(n_adult_entity_id))

    n_children_entity_id = client.model.add_entity(
        app_id, app_version, name="n_children")
    print("flightEntityId {} added.".format(n_children_entity_id))

    budget_entity_id = client.model.add_entity(
        app_id, app_version, name="budget")
    print("flightEntityId {} added.".format(budget_entity_id))

    seat_entity_id = client.model.add_entity(
        app_id, app_version, name="seat")
    print("flightEntityId {} added.".format(seat_entity_id))

def add_intents(app_id, app_version):
    '''
        Création des intention :
            - BookFlight
            Parameters:
                    app_id : ID de l\'application 
                    app_version : version de l\'application LUIS
            Returns:
                    None
    '''
    intent_id = client.model.add_intent(app_id, app_version, "BookFlight")
    print("Intent BookFlight {} added.".format(intent_id))


In [36]:
add_entities(app_id=app_id,app_version=app_version)

In [20]:
add_intents(app_id=app_id,app_version=app_version)

Intent BookFlight 024458ec-3211-40d5-a41f-85f83f07cd04 added.


In [7]:
luis_transformed_data[0]

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intent_name': 'BookFlight',
 'entity_labels': [{'entity_name': 'dst_city',
   'start_char_index': 27,
   'end_char_index': 35},
  {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48},
  {'entity_name': 'str_date', 'start_char_index': 52, 'end_char_index': 77},
  {'entity_name': 'budget', 'start_char_index': 117, 'end_char_index': 121}]}

In [6]:
train_data = luis_transformed_data[:-99]
test_data = luis_transformed_data[-99:]

In [33]:
for i in range(0, len(train_data), 100):
    j = (i + 100)
    if j > len(luis_transformed_data):
        j = len(luis_transformed_data)
    # On ajoute 100 par 100 car les requêtes sont bloqué au dela de 100
    client.examples.batch(app_id,app_version,train_data[i:j])

In [37]:
client.train.train_version(app_id, app_version)
waiting = True
while waiting:
    info = client.train.get_status(app_id, app_version)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [38]:
client.apps.update_settings(app_id, is_public=True)

responseEndpointInfo = client.apps.publish(app_id, app_version, is_staging=False)

In [1]:
endpoint = responseEndpointInfo.endpoint_url + \
"?subscription-key=" + AUTHORING_KEY_LUIS + "&q="
print("Your app is published. You can now go to test it on\n{}".format(endpoint))

### Test de prédiction du modèle déployé

In [23]:
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient

PREDICTION_KEY = os.environ["RUNTIME_KEY_LUIS"]
PREDICTION_ENDPOINT = os.environ["RUNTIME_ENDPOINT_LUIS"]
clientRuntime = LUISRuntimeClient(PREDICTION_ENDPOINT, CognitiveServicesCredentials(PREDICTION_KEY))

In [24]:
request = { "query" : "book a flight from paris to berlin for 2020 between 21th september and 2nd october" }

# Note be sure to specify, using the slot_name parameter, whether your application is in staging or production.
response = clientRuntime.prediction.get_slot_prediction(app_id=app_id, slot_name="production", prediction_request=request)

In [34]:
response.prediction.entities

{'or_city': ['paris'],
 'dst_city': ['berlin'],
 'str_date': ['september'],
 'end_date': ['2nd october']}

In [41]:
for intent in response.prediction.intents.keys():
    print(intent)

BookFlight


In [35]:
import json
print("Top intent: {}".format(response.prediction.top_intent))
print("Intents: ")

for intent in response.prediction.intents:
	print("\t{}".format (json.dumps (intent)))

print("Entities: {}".format (response.prediction.entities))

Top intent: BookFlight
Intents: 
	"BookFlight"
Entities: {'or_city': ['paris'], 'dst_city': ['berlin'], 'str_date': ['september'], 'end_date': ['2nd october']}


In [5]:
import requests
import json

def predict(query):
    response = requests.get(endpoint+query)
    response = response.json()
    print("Top intent: {}".format(response["topScoringIntent"]["intent"]))
    print("Entities: ")

    for entity in response["entities"]:
        data = entity
        key = data["type"]
        value = data["entity"]
        print("{\'",key,"\' : \'",value,"\'}")

In [6]:
test = "we're from punta cana. i'd like to bring them while they're still out of school, sometime between wednesday september 14 and tuesday october 4th would be great."
# test = "book a flight from paris to berlin for 2020 between 21th september and 2nd october"

In [7]:
predict(test)

Top intent: BookFlight
Entities: 
{' end_date ' : ' october 4th '}
{' or_city ' : ' punta cana . '}
{' str_date ' : ' wednesday september 14 '}


## Test de l'object LuiApp permettant de simplifier les interactions avec LUIS Azure

In [4]:
from luis_module.luis_utility import LuisApp
import os

AUTHORING_KEY_LUIS = os.environ["AUTHORING_KEY_LUIS"]
AUTHORING_ENDPOINT_LUIS = os.environ["AUTHORING_ENDPOINT_LUIS"]

luis_app = LuisApp(AUTHORING_ENDPOINT_LUIS,AUTHORING_KEY_LUIS)
luis_app.get_app("8958f321-de6e-4cee-bf26-e2468b5f381f")

In [5]:
luis_app.predict("we're from punta cana. i'd like to bring them while they're still out of school, sometime between wednesday september 14 and tuesday october 4th would be great.")

{'end_date': 'october 4th',
 'or_city': 'punta cana .',
 'str_date': 'wednesday september 14'}

In [3]:
from luis_module.get_data import LuisData 

luis_data = LuisData("./data/frames.json")
luis_data.load_data()

luis_transformed_data = luis_data.transform_data()

In [4]:
train_data = luis_transformed_data[:-99]
test_data = luis_transformed_data[-99:]

In [8]:
all_score, score = luis_app.evaluate(test_data)

In [12]:
print("accuracy score :",round(score,2)) 

accuracy score : 0.61
